# Targeted SIB Training

In [1]:
%reload_ext autoreload
%autoreload 2

In [2]:
from transformers import (
    AutoModelForSequenceClassification, 
    AutoTokenizer, 
    Trainer, 
    TrainingArguments, 
    TrainerCallback, 
    EarlyStoppingCallback
)
from sklearn.metrics import accuracy_score, precision_recall_fscore_support
from transformers.trainer_callback import TrainerControl
from datasets import load_dataset
import torch
import pandas as pd
from torch.utils.data import DataLoader
from transforms import TextMix, SentMix, WordMix, SibylCollator

device = torch.device('cuda') if torch.cuda.is_available() else torch.device('cpu')

In [3]:
def tokenize_fn(text):
    return tokenizer(text, padding=True, truncation=True, max_length=250, return_tensors='pt')

def tokenize(batch):
    return tokenizer(batch['text'], padding=True, truncation=True, max_length=250)

def acc_at_k(y_true, y_pred, k=2):
    y_true = torch.tensor(y_true) if type(y_true) != torch.Tensor else y_true
    y_pred = torch.tensor(y_pred) if type(y_pred) != torch.Tensor else y_pred
    total = len(y_true)
    y_weights, y_idx = torch.topk(y_true, k=k, dim=-1)
    out_weights, out_idx = torch.topk(y_pred, k=k, dim=-1)
    correct = torch.sum(torch.eq(y_idx, out_idx) * y_weights)
    acc = correct / total
    return acc.item()

def CEwST_loss(logits, target, reduction='mean'):
    """
    Cross Entropy with Soft Target (CEwST) Loss
    :param logits: (batch, *)
    :param target: (batch, *) same shape as logits, each item must be a valid distribution: target[i, :].sum() == 1.
    """
    logprobs = torch.nn.functional.log_softmax(logits.view(logits.shape[0], -1), dim=1)
    batchloss = - torch.sum(target.view(target.shape[0], -1) * logprobs, dim=1)
    if reduction == 'none':
        return batchloss
    elif reduction == 'mean':
        return torch.mean(batchloss)
    elif reduction == 'sum':
        return torch.sum(batchloss)
    else:
        raise NotImplementedError('Unsupported reduction mode.')
        
def compute_metrics(pred):
    preds, labels = pred
    if len(labels.shape) > 1: 
        acc = acc_at_k(labels, preds, k=2)
        return { 'accuracy': acc }        
    else:
        acc = accuracy_score(labels, preds.argmax(-1))
        return { 'accuracy': acc }        

class TargetedTrainer(Trainer):
    def compute_loss(self, model, inputs, return_outputs=False):
        labels = inputs.pop("labels")
        outputs = model(**inputs)
        logits = outputs[0]
        if len(labels.shape) > 1: 
            loss = CEwST_loss(logits, labels)
        else:
            loss = torch.nn.functional.cross_entropy(logits, labels)
        if return_outputs:
            return loss, outputs
        return loss

class TargetedMixturesCallback(TrainerCallback):
    """
    A callback that calculates a confusion matrix on the validation
    data and returns the most confused class pairings.
    """
    def __init__(self, dataloader, device):
        self.dataloader = dataloader
        self.device = device
        
    def on_evaluate(self, args, state, control, model, tokenizer, **kwargs):
        cnf_mat = self.get_confusion_matrix(model, tokenizer, self.dataloader)
        new_targets = self.get_most_confused_per_class(cnf_mat)
        print("New targets:", new_targets)
        control = TrainerControl
        control.new_targets = new_targets
        if state.global_step < state.max_steps:
            control.should_training_stop = False
        else:
            control.should_training_stop = True
        return control
        
    def get_confusion_matrix(self, model, tokenizer, dataloader, normalize=True):
        n_classes = max(dataloader.dataset['label']) + 1
        confusion_matrix = torch.zeros(n_classes, n_classes)
        with torch.no_grad():
            for batch in iter(self.dataloader):
                data, targets = batch['text'], batch['label']
                data = tokenizer(data, padding=True, truncation=True, max_length=250, return_tensors='pt')
                input_ids = data['input_ids'].to(self.device)
                attention_mask = data['attention_mask'].to(self.device)
                targets = targets.to(self.device)
                outputs = model(input_ids, attention_mask=attention_mask).logits
                preds = torch.argmax(outputs, dim=1).cpu()
                for t, p in zip(targets.view(-1), preds.view(-1)):
                    confusion_matrix[t.long(), p.long()] += 1    
            if normalize:
                confusion_matrix = confusion_matrix / confusion_matrix.sum(dim=0)
        return confusion_matrix

    def get_most_confused_per_class(self, confusion_matrix):
        idx = torch.arange(len(confusion_matrix))
        cnf = confusion_matrix.fill_diagonal_(0).max(dim=1)[1]
        return torch.stack((idx, cnf)).T.tolist()

class TargetedMixturesCollator:
    def __init__(self, 
                 tokenize_fn, 
                 transform, 
                 transform_prob=1.0, 
                 target_pairs=[], 
                 target_prob=1.0, 
                 num_classes=2):
        
        self.tokenize_fn = tokenize_fn
        self.transform = transform
        self.transform_prob = transform_prob
        self.target_pairs = target_pairs
        self.target_prob = target_prob
        self.num_classes = num_classes
        print("TargetedMixturesCollator initialized with {}".format(transform.__class__.__name__))
        
    def __call__(self, batch):
        text = [x['text'] for x in batch]
        labels = [x['label'] for x in batch]
        batch = (text, labels)
        if torch.rand(1) < self.transform_prob:
            batch = self.transform(
                batch, 
                self.target_pairs,   
                self.target_prob,
                self.num_classes
            )
        text, labels = batch
        labels = torch.tensor(labels)
        if len(labels.shape) == 1:
            labels = torch.nn.functional.one_hot(labels, num_classes=self.num_classes)
        batch = self.tokenize_fn(text)
        batch['labels'] = labels
        batch.pop('idx', None)
        batch.pop('label', None)
        return batch
    
class DefaultCollator:
    def __init__(self):
        pass
    def __call__(self, batch):
        return torch.utils.data.dataloader.default_collate(batch)

In [4]:
MODEL_NAMES = ['bert-base-uncased', 'roberta-base', 'xlnet-base-cased']
# ts = ['ORIG', 'INV', 'SIB', 'INVSIB', 'TextMix', 'SentMix', 'WordMix']
ts = ['SIB', 'INVSIB', 'TextMix', 'SentMix', 'WordMix']

In [5]:
results = []

for MODEL_NAME in MODEL_NAMES:
        
    for t in ts: 
        
        transform = None
        num_sampled_INV = 0
        num_sampled_SIB = 0
        label_type = "soft"
        
        if t == "INV":
            num_sampled_INV = 2
            label_type = "hard"
        elif t == "SIB":
            num_sampled_SIB = 2
        elif t == 'INVSIB':
            num_sampled_INV = 1
            num_sampled_SIB = 1
            label_type = None
        elif t == "TextMix":
            transform = TextMix()
        elif t == "SentMix":
            transform = SentMix()
        elif t == "WordMix":
            transform = WordMix()
        
        checkpoint = './results/' + MODEL_NAME + '-SibylCollator-' + t
        
        tokenizer = AutoTokenizer.from_pretrained(MODEL_NAME)
        model = AutoModelForSequenceClassification.from_pretrained(MODEL_NAME, num_labels=5).to(device)

        dataset = load_dataset('ag_news', split='train') 
        dataset_dict = dataset.train_test_split(
            test_size = 0.05,
            train_size = 0.95,
            shuffle = True
        )
        train_dataset = dataset_dict['train']
        eval_dataset = dataset_dict['test']

        test_dataset = load_dataset('ag_news', split='test') 
        test_dataset.rename_column_('label', 'labels')
        test_dataset = test_dataset.map(tokenize, batched=True, batch_size=len(test_dataset))
        test_dataset.set_format('torch', columns=['input_ids', 'attention_mask', 'labels'])
        
        train_batch_size = 6
        eval_batch_size  = 32
        num_epoch = 20
        gradient_accumulation_steps = 1
        max_steps = int((len(train_dataset) * num_epoch / gradient_accumulation_steps) / train_batch_size)

#         tmcb = TargetedMixturesCallback(
#             dataloader=DataLoader(eval_dataset, batch_size=32),
#             device=device
#         )
        escb = EarlyStoppingCallback(
            early_stopping_patience=10
        )
#         tmc = TargetedMixturesCollator(
#             tokenize_fn=tokenize_fn, 
#             transform=t, 
#             transform_prob=0.5,
#             target_pairs=[],
#             target_prob=0.5,
#             num_classes=4
#         )
        sibyl_collator = SibylCollator( 
            tokenize_fn=tokenize_fn, 
            transform=transform, 
            num_sampled_INV=num_sampled_INV, 
            num_sampled_SIB=num_sampled_SIB, 
            task_type="topic", 
            tran_type=None, 
            label_type=label_type,
            one_hot=label_type == "soft",
            transform_prob=0.5,
            target_pairs=[],
            target_prob=0.0,
            reduce_mixed=True,
            num_classes=4
        )

        training_args = TrainingArguments(\
            output_dir=checkpoint,
            overwrite_output_dir=True,
            max_steps=max_steps,
            save_steps=int(max_steps / 10),
            save_total_limit=1,
            per_device_train_batch_size=train_batch_size,
            per_device_eval_batch_size=eval_batch_size,
            gradient_accumulation_steps=gradient_accumulation_steps, 
            warmup_steps=int(max_steps / 10),
            weight_decay=0.01,
            logging_dir='./logs',
            logging_steps=5000,
            logging_first_step=True,
            load_best_model_at_end=True,
            metric_for_best_model="accuracy",
            greater_is_better=True,
            evaluation_strategy="steps",
            remove_unused_columns=False
        )

        trainer = TargetedTrainer(
            model=model, 
            tokenizer=tokenizer,
            args=training_args,
            compute_metrics=compute_metrics,                  
            train_dataset=train_dataset,         
            eval_dataset=eval_dataset,
            data_collator=sibyl_collator if t != "ORIG" else DefaultCollator(),
            callbacks=[escb] # [tmcb, escb]
        )

        trainer.train()

        # test with ORIG data
        trainer.eval_dataset = test_dataset
        trainer.data_collator = DefaultCollator()
        # trainer.remove_callback(tmcb)

        out_orig = trainer.evaluate()
        out_orig['run'] = checkpoint
        out_orig['test'] = "ORIG"
        print('ORIG for {}\n{}'.format(checkpoint, out_orig))

        results.append(out_orig)

Some weights of the model checkpoint at bert-base-uncased were not used when initializing BertForSequenceClassification: ['cls.predictions.bias', 'cls.predictions.transform.dense.weight', 'cls.predictions.transform.dense.bias', 'cls.predictions.decoder.weight', 'cls.seq_relationship.weight', 'cls.seq_relationship.bias', 'cls.predictions.transform.LayerNorm.weight', 'cls.predictions.transform.LayerNorm.bias']
- This IS expected if you are initializing BertForSequenceClassification from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing BertForSequenceClassification from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
Some weights of BertForSequenceClassification were not initialized from the model checkpoint at


SibylCollator initialized with num_sampled_INV=0 and num_sampled_SIB=2


Step,Training Loss,Validation Loss,Accuracy,Runtime,Samples Per Second
5000,0.692900,0.408098,0.897500,12.086000,99.289000
10000,0.371800,0.363069,0.916667,11.517400,104.190000
15000,0.355200,0.404053,0.907500,9.974200,120.310000
20000,0.371100,0.342812,0.922500,10.254100,117.027000
25000,0.346600,0.361725,0.920000,11.025600,108.837000
30000,0.361200,0.370168,0.909167,11.118700,107.927000
35000,0.360500,0.475786,0.905000,10.412100,115.250000
40000,0.382800,0.496682,0.887500,11.797800,101.713000
45000,0.365200,0.412427,0.919167,10.352000,115.920000
50000,0.368800,0.576396,0.905000,11.630600,103.176000


ORIG for ./results/bert-base-uncased-SibylCollator-SIB
{'eval_loss': 0.2793702483177185, 'eval_accuracy': 0.9313157894736842, 'eval_runtime': 113.758, 'eval_samples_per_second': 66.808, 'epoch': 3.54, 'run': './results/bert-base-uncased-SibylCollator-SIB', 'test': 'ORIG'}


Some weights of the model checkpoint at bert-base-uncased were not used when initializing BertForSequenceClassification: ['cls.predictions.bias', 'cls.predictions.transform.dense.weight', 'cls.predictions.transform.dense.bias', 'cls.predictions.decoder.weight', 'cls.seq_relationship.weight', 'cls.seq_relationship.bias', 'cls.predictions.transform.LayerNorm.weight', 'cls.predictions.transform.LayerNorm.bias']
- This IS expected if you are initializing BertForSequenceClassification from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing BertForSequenceClassification from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
Some weights of BertForSequenceClassification were not initialized from the model checkpoint at

...

ORIG for ./results/xlnet-base-cased-SibylCollator-SentMix
{'eval_loss': 0.32802721858024597, 'eval_accuracy': 0.9264473684210527, 'eval_runtime': 246.4713, 'eval_samples_per_second': 30.835, 'epoch': 3.54, 'run': './results/xlnet-base-cased-SibylCollator-SentMix', 'test': 'ORIG'}


Some weights of the model checkpoint at xlnet-base-cased were not used when initializing XLNetForSequenceClassification: ['lm_loss.weight', 'lm_loss.bias']
- This IS expected if you are initializing XLNetForSequenceClassification from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing XLNetForSequenceClassification from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
Some weights of XLNetForSequenceClassification were not initialized from the model checkpoint at xlnet-base-cased and are newly initialized: ['sequence_summary.summary.weight', 'sequence_summary.summary.bias', 'logits_proj.weight', 'logits_proj.bias']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions a


SibylCollator initialized with WordMix


Step,Training Loss,Validation Loss,Accuracy,Runtime,Samples Per Second
5000,0.656700,0.601203,0.861667,19.431300,61.756000
10000,0.445700,0.449367,0.879167,19.561800,61.344000
15000,0.424400,0.357637,0.918333,17.811200,67.373000
20000,0.401600,0.354163,0.926667,17.347000,69.176000
25000,0.401800,0.417600,0.895000,17.396400,68.980000
30000,0.410500,0.502811,0.887500,18.857200,63.636000
35000,0.428500,0.444306,0.905000,16.856000,71.191000
40000,0.479000,1.129131,0.724167,19.036200,63.038000
45000,0.520800,0.739449,0.848333,17.801600,67.410000
50000,0.616800,0.729014,0.844167,18.960200,63.290000


ORIG for ./results/xlnet-base-cased-SibylCollator-WordMix
{'eval_loss': 0.33383435010910034, 'eval_accuracy': 0.9285526315789474, 'eval_runtime': 247.9231, 'eval_samples_per_second': 30.655, 'epoch': 3.54, 'run': './results/xlnet-base-cased-SibylCollator-WordMix', 'test': 'ORIG'}


In [6]:
df = pd.DataFrame(results)

In [7]:
df.to_csv('train_AG_NEWS_SibylCollator.csv')

In [8]:
df.to_clipboard(excel=True)

In [10]:
df

,eval_loss,eval_accuracy,eval_runtime,eval_samples_per_second,epoch,run,test
0,0.279370,0.931316,113.7580,66.808,3.54,./results/bert-base-uncased-SibylCollator-SIB,ORIG
1,0.293198,0.927763,112.8770,67.330,3.28,./results/bert-base-uncased-SibylCollator-INVSIB,ORIG
2,0.325988,0.927895,113.6751,66.857,3.54,./results/bert-base-uncased-SibylCollator-TextMix,ORIG
3,0.325761,0.925000,113.0304,67.239,3.28,./results/bert-base-uncased-SibylCollator-SentMix,ORIG
4,0.274907,0.926184,112.7674,67.395,3.28,./results/bert-base-uncased-SibylCollator-WordMix,ORIG
5,0.353707,0.922105,110.1731,68.982,3.28,./results/roberta-base-SibylCollator-SIB,ORIG
6,0.355798,0.928289,107.7642,70.524,4.80,./results/roberta-base-SibylCollator-INVSIB,ORIG
7,0.274713,0.931184,109.0320,69.704,3.28,./results/roberta-base-SibylCollator-TextMix,ORIG
8,0.332986,0.921579,110.0170,69.080,3.28,./results/roberta-base-SibylCollator-SentMix,ORIG
9,0.290559,0.927763,111.5945,68.104,3.54,./results/roberta-base-SibylCollator-WordMix,ORIG
